In [5]:

import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
import pandas as pd
import re
from tqdm import tqdm
import streamlit as st
import sqlite3


# In[2]:


def extract_number(text, return_type=float):
    """テキスト内から数字を取り出して返す（float or int）

    Parameters:
    ----------
    text : str
        数字が入ったテキスト
    return_type : type
        返す値の型。引数なしではfloat型となる

    Returns:
    ----------
    型：return_typeで選択した型
        textから取り出した数字を返す
    """
    matched_text = re.search(r'\d+(\.\d+)?', text)
    if matched_text:
        number = matched_text.group()
        if return_type == int:
            return int(float(number))  # floatへの変換後にintへ変換
        else:
            return float(number)
    else:
        return 0 if return_type == int else 0.0


# In[15]:


# Airdoorデータ取得
d_list = []
url = 'https://airdoor.jp/list?si=d-131083&p={}'

for i in tqdm(range(1,15)):
    target_url = url.format(i)
    r = requests.get(target_url)
    time.sleep(1) # 1秒ずつ
    soup = BeautifulSoup(r.text,"html.parser")
    contents = soup.find_all('div', {'class': 'PropertyPanel_propertyPanel__8oJ13'}) or None
    for content in contents:
        # タイトル
        title = content.find('div', {'class': 'PropertyPanelBuilding_buildingTitle__tuPqN'}).get_text(strip=True) or None
        # 住所
        building_info = content.find_all('div', {'class': 'PropertyPanelBuilding_buildingInformationSection__deSLp'})
        address = building_info[0].find('p', {'class': 'is-mt5'}).get_text(strip=True) or None
        access = ', '.join(p.get_text() for p in building_info[0].find_all('p', {'class': False})) or None
        # 築年数、総階数
        p_tags = building_info[1].find_all('p')
        age = re.search(r'\((.*?)\)', p_tags[0].get_text()).group(1) or '築0年'
        story = p_tags[1].get_text(strip=True)
        # 階数、間取り、面積
        roomItems = content.findAll('a', {'class': 'PropertyPanelRoom_roomItem__95jRr'})
        for roomItem in roomItems:
            p_tag_text = roomItem.find('span', {'class': 'is-ml5'}).get_text(strip=True)
            room_number, madori, menseki, hogaku = [part.strip() for part in p_tag_text.split('/')]
            # 階数
            floor = re.findall(r'\d+', room_number)[0][:-2] if re.findall(r'\d+', room_number) and len(re.findall(r'\d+', room_number)[0]) > 2 else '1'
            # 家賃、管理費
            div_text = roomItem.find('div', {'class': 'PropertyPanelRoom_rentPrice__XdPUp'}).text
            fee = div_text.split()[0].replace(',', '') or '0円'
            management_fee = div_text.split()[1].replace(',', '') or '0円'
            # 敷金、礼金
            div = roomItem.find('div', {'class': 'PropertyPanelRoom_initialPrices__d90C3'})
            deposit = div.find_all('li')[0].get_text(strip=True) or '0円'
            gratuity = div.find_all('li')[1].get_text(strip=True) or '0円'
            d = {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'story': story,
                'floor': floor,
                'room_number': room_number,
                'fee': fee,
                'management_fee': management_fee,
                'deposit': deposit,
                'gratuity': gratuity,
                'madori': madori,
                'menseki': menseki,
            }
            d_list.append(d)
df_airdoor = pd.DataFrame(d_list)


# In[16]:


df_airdoor['title'] = df_airdoor['title'].str.replace(r'【.*?】', '', regex=True)
df_airdoor['fee'] = df_airdoor['fee'].apply(extract_number)/10000
df_airdoor['management_fee'] = df_airdoor['management_fee'].apply(extract_number)/10000
df_airdoor['deposit'] = df_airdoor['deposit'].apply(lambda x: "0円" if x in ["無料"] else x).apply(extract_number)
df_airdoor['gratuity'] = df_airdoor['gratuity'].apply(lambda x: "0円" if x in ["無料"] else x).apply(extract_number)
df_airdoor['age'] = df_airdoor['age'].apply(lambda x: "築1年" if x in ["新築", "築0年"] else x).apply(extract_number)
df_airdoor['story'] = df_airdoor['story'].apply(extract_number)
df_airdoor['floor'] = df_airdoor['floor'].apply(extract_number)
df_airdoor['menseki'] = df_airdoor['menseki'].apply(extract_number)


# In[17]:


# accessを取得し、「路線」「駅名」「徒歩分数」に分割し、それぞれ「access1_1」「access1_2」「access1_3」に格納する。アクセスは最大2件まで取得する
# df_airdoorにカラム追加
for i in range(1, 3):
    for j in range(1, 4):
        df_airdoor[f'access{i}_{j}'] = ''
df_airdoor.head()
# 行ごとにテキストを分解してカラムに格納
for index, row in df_airdoor.iterrows():
    accesses = row['access'].split(',')[:3] # アクセス情報をコンマで分割し、最大3つまで取得
    for i, access in enumerate(accesses, start=1):
        match = re.match(r'(.+?)\s+(.+?)\s+徒歩(\d+)分', access.strip()) # 正規表現でテキストを解析
        if match:
            df_airdoor.at[index, f'access{i}_1'] = match.group(1)
            df_airdoor.at[index, f'access{i}_2'] = match.group(2)
            df_airdoor.at[index, f'access{i}_3'] = match.group(3)


# In[26]:


# 複数のスクレイピングデータを統合、重複物件を排除する場合はここで
df_scraped = df_airdoor

# 共通データを付与
from datetime import datetime
current_time = datetime.now().strftime('%Y-%m-%d %H:%M')
df_scraped['scraped_date_time'] = current_time
df_scraped['daily_decreased_room'] = None
df_scraped['weekly_decreased_room'] = None
df_scraped['evaluation_score'] = None


100%|██████████| 14/14 [00:38<00:00,  2.74s/it]

          title      address                                    access   age  \
0   キャナルスクウェア豊洲  東京都江東区豊洲１丁目   東京メトロ有楽町線 月島駅 徒歩17分, 東京メトロ有楽町線 豊洲駅 徒歩9分  15.0   
1   キャナルスクウェア豊洲  東京都江東区豊洲１丁目   東京メトロ有楽町線 月島駅 徒歩17分, 東京メトロ有楽町線 豊洲駅 徒歩9分  15.0   
2   キャナルスクウェア豊洲  東京都江東区豊洲１丁目   東京メトロ有楽町線 月島駅 徒歩17分, 東京メトロ有楽町線 豊洲駅 徒歩9分  15.0   
3   プライムメゾン清澄白河  東京都江東区平野２丁目  都営大江戸線 清澄白河駅 徒歩10分, 東京メトロ東西線 門前仲町駅 徒歩11分   1.0   
4   プライムメゾン清澄白河  東京都江東区平野２丁目  都営大江戸線 清澄白河駅 徒歩10分, 東京メトロ東西線 門前仲町駅 徒歩11分   1.0   

   story  floor room_number   fee  management_fee  deposit  ...  access1_1  \
0   23.0    8.0       816号室  18.3             0.8     18.3  ...  東京メトロ有楽町線   
1   23.0    3.0       316号室  17.3             0.8     17.3  ...  東京メトロ有楽町線   
2   23.0    3.0       314号室  17.3             0.8     17.3  ...  東京メトロ有楽町線   
3    7.0    2.0      0203号室  11.6             1.0      0.0  ...     都営大江戸線   
4    7.0    2.0      0201号室  17.6             1.0      0.0  ...     都営大江戸線   

  access1_2  access1_3  access2_1 access2_2 access

In [6]:
# データを読み込んでdf_dbに格納
db_path = 'techone_2.db'
conn = sqlite3.connect(db_path)
query = 'SELECT * FROM techone_db;'
df_db = pd.read_sql_query(query, conn)
conn.close()

                   title      address  \
0      プレール・ドゥーク木場公園ＷＥＳＴ  東京都江東区平野３丁目   
1            プライムメゾン清澄白河  東京都江東区平野２丁目   
2            プライムメゾン清澄白河  東京都江東区平野２丁目   
3            プライムメゾン清澄白河  東京都江東区平野２丁目   
4               リビオメゾン木場  東京都江東区木場６丁目   
...                  ...          ...   
1743               ペルル亀戸  東京都江東区亀戸４丁目   
1744          シーズンフラッツ木場  東京都江東区木場３丁目   
1745          シーズンフラッツ木場  東京都江東区木場３丁目   
1746          シーズンフラッツ木場  東京都江東区木場３丁目   
1747   S-RESIDENCE亀戸lume  東京都江東区大島２丁目   

                                        access  age  story  floor room_number  \
0     東京メトロ半蔵門線 清澄白河駅 徒歩8分, 都営大江戸線 清澄白河駅 徒歩11分  1.0    8.0    6.0       603号室   
1     都営大江戸線 清澄白河駅 徒歩10分, 東京メトロ東西線 門前仲町駅 徒歩11分  1.0    7.0    2.0      0203号室   
2     都営大江戸線 清澄白河駅 徒歩10分, 東京メトロ東西線 門前仲町駅 徒歩11分  1.0    7.0    2.0      0201号室   
3     都営大江戸線 清澄白河駅 徒歩10分, 東京メトロ東西線 門前仲町駅 徒歩11分  1.0    7.0    5.0      0507号室   
4       東京メトロ東西線 木場駅 徒歩4分, 東京メトロ東西線 東陽町駅 徒歩15分  1.0   10.0    8.0       802号室   
...                  

In [9]:
# df_dbのうち、最新のもののみをdf_db_1に格納
last_datetime = df_db['scraped_date_time'].max()
df_db_1 = df_db[df_db['scraped_date_time']==last_datetime]

(436, 23)

In [10]:
df_marged = pd.merge(df_db_1, df_scraped, on=['title', 'address', 'room_number'], how='left', indicator=True)

In [11]:
df_unique = df_marged[df_marged['_merge']=='left_only']
df_unique.shape

(0, 44)

In [12]:
df_unique[df_unique['title'].str.contains('シーズンフラッツ木場')]

,title,address,access_x,age_x,story_x,floor_x,room_number,fee_x,management_fee_x,deposit_x,...,access1_2_y,access1_3_y,access2_1_y,access2_2_y,access2_3_y,scraped_date_time_y,daily_decreased_room_y,weekly_decreased_room_y,evaluation_score_y,_merge


In [ ]:
# df_dbの最新レコードに、daily_decreased_roomの値を追加。df_uniqueのレコード数をtitle、addressをキーにカウント → df_dbと左結合 → df_dbの最新レコードにのみ、daily_decreased_roomにcountを代入

# ステップ 1: 'title' と 'address' の組み合わせごとにレコード数をカウント
count_df_unique = df_unique.groupby(['title', 'address']).size().reset_index(name='count')

# ステップ 2: このカウントを df_db にマージ（左結合）
df_db_merged = pd.merge(df_db, count_df_unique, on=['title', 'address'], how='left')

# ステップ 3: df_db_mergedのうち、'scraped_date_time'が最大のものにのみ、'count'の値を'daily_decreased_room' に追加
last_datetime = df_db_merged['scraped_date_time'].max()
df_db_merged.loc[df_db_merged['scraped_date_time'] == last_datetime, 'daily_decreased_room'] = df_db_merged['count']

# 必要に応じて、マージした後のデータフレームから不要なカラムを削除
df_db_merged.drop(columns=['count'], inplace=True)

In [36]:
df_db_combined_1 = df_db_merged
# 'title', 'address', 'scraped_date_time'をキーに重複行を削除
df_db_combined_1_unique = df_db_combined_1.drop_duplicates(subset=['title', 'address', 'scraped_date_time'])

In [41]:
# 'weekly_decreased_room'に、'title', 'address'が同値の'scraped_date_time'の合計を代入する
df_db_combined_1_unique.loc[:, 'weekly_decreased_room'] = df_db_combined_1_unique.groupby(['title', 'address'])['daily_decreased_room'].transform('sum')

In [61]:
# 元のdf_dbに daily_decreased_room と weekly_decreased_room を追加する
df_db_merged = pd.merge(df_db, df_db_combined_1_unique[['title', 'address', 'scraped_date_time', 'daily_decreased_room', 'weekly_decreased_room']],on=['title', 'address', 'scraped_date_time'], how='left', suffixes=('', '_1'))
df_db['daily_decreased_room'] = df_db_merged['daily_decreased_room_1']
df_db['weekly_decreased_room'] = df_db_merged['weekly_decreased_room_1']

# weekly_decreased_roomの値に応じて、evaluation_scoreの値を代入
import numpy as np

# 条件を設定
conditions = [
    df_db['weekly_decreased_room'] < 5,  # weekly_decreased_roomが5より小さい
    df_db['weekly_decreased_room'].between(5, 10),  # weekly_decreased_roomが5〜10の間
    df_db['weekly_decreased_room'] > 10  # weekly_decreased_roomが10より大きい
]

# 各条件に対する値を設定
values = [1, 2, 3]

# numpy.selectを使用して条件に基づく値を設定
df_db['evaluation_score'] = np.select(conditions, values, default=np.nan)

In [63]:
# scraped_date_timeが最大のものだけ取り出す。これが最終的に欲しかったdfとなる
last_datetime = df_db['scraped_date_time'].max()
df_db_with_value = df_db[df_db['scraped_date_time']==last_datetime]

In [65]:
conn = sqlite3.connect('techone_2.db')

# DataFrameのインデックスをカラムに変換
df_with_index = df_db_with_value.reset_index().rename(columns={'index': 'index_column'})

for index, row in df_with_index.iterrows():
    sql = """UPDATE techone_db
            SET daily_decreased_room = ?,
                weekly_decreased_room = ?,
                evaluation_score = ?
            WHERE title = ? AND address = ? AND room_number = ? AND scraped_date_time = ?"""
    conn.execute(sql, (row['daily_decreased_room'], row['weekly_decreased_room'], row['evaluation_score'], row['title'], row['address'], row['room_number'], row['scraped_date_time']))

# 変更をコミット
conn.commit()

# 接続を閉じる
conn.close()


In [ ]:
# SQLiteデータベースへの接続
conn = sqlite3.connect('/Users/ryosukeinoue/Library/CloudStorage/GoogleDrive-ryosuke.inoue0314@gmail.com/マイドライブ/00_本データ/31_Tech0/Step3/techone_new/techone_2.db')

# DataFrameをSQLiteデータベースにインポート
df_airdoor.to_sql('techone_scraped', conn, if_exists='replace', index=False)

# 接続を閉じる
conn.close()

In [60]:
# SQLiteデータベースへの接続
conn = sqlite3.connect('/Users/ryosukeinoue/Library/CloudStorage/GoogleDrive-ryosuke.inoue0314@gmail.com/マイドライブ/00_本データ/31_Tech0/Step3/techone_new/techone_2.db')

# DataFrameをSQLiteデータベースにインポート
df_scraped.to_sql('techone_db', conn, if_exists='append', index=False)

# 接続を閉じる
conn.close()